<a href="https://colab.research.google.com/github/soumitrapy/docs/blob/main/torrent2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Downloader
%%capture
import warnings
warnings.filterwarnings("ignore")

!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent
import time
from IPython.display import display
import ipywidgets as widgets

import libtorrent as lt
ses = lt.session()
ses.listen_on(6881, 6891)


class Downloader:
  def __init__(self, path="/content"):
    self.params = {"save_path": path}
    self.links = []
    self.state_str = [
        "queued",
        "checking",
        "downloading metadata",
        "downloading",
        "finished",
        "seeding",
        "allocating",
        "checking fastresume",
    ]
    self.layout = widgets.Layout(width="auto")
    self.style = {"description_width": "initial"}

  def download(self, k=None):
    if k is None:
      link = input("Enter Magnet Link Or Type Enter: ")
      if len(link)>1:
        self.links.append(link)
      k = -1

    link = self.links[k]
    download = lt.add_magnet_uri(ses, link, self.params)
    bar = widgets.FloatSlider(step=0.01,
                              disabled=True,
                              layout=self.layout,
                              style=self.style
                              )

    display(bar)

    while self.links:
      next_shift = 0
      if not download.is_seed():
          s = download.status()

          bar.description = " ".join(
              [
                  download.name()[:30],
                  str(s.download_rate / 1000),
                  "kB/s",
                  self.state_str[s.state],
              ]
          )
          bar.value = s.progress * 100
      else:
          next_shift -= 1
          ses.remove_torrent(download)
          self.links.pop(k)
          print("="*10+"Download Complete"+"="*10)

  def clear(self):
    self.links.clear()

d = Downloader()
#d.download(-1)
#d.links

In [ ]:
d.download()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')